# Los Angeles Music Composer Maker (ver. 0.5)

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

WARNING: This complete implementation is a functioning model of the Artificial Intelligence. Please excercise great humility, care, and respect. https://www.nscai.gov/

***

#### Project Los Angeles

#### Tegridy Code 2022

***

# GPU check

In [ ]:
!nvidia-smi

# Setup environment

In [ ]:
!git clone https://github.com/asigalov61/tegridy-tools

In [ ]:
#!pip install torch
!pip install einops
!pip install torch-summary
#!pip install sklearn
!pip install tqdm
#!pip install matplotlib

In [ ]:
# Load modules and make data dir

print('Loading modules...')

import os
import pickle
import random
import secrets
import tqdm

import torch
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

import matplotlib.pyplot as plt

from torchsummary import summary
from sklearn import metrics

%cd /notebooks/tegridy-tools/tegridy-tools/

import TMIDIX

%cd /notebooks/tegridy-tools/tegridy-tools/LWA-Transformer

from lwa_transformer import *

%cd /notebooks/

if not os.path.exists('/notebooks/INTS'):
    os.makedirs('/notebooks/INTS')

print('Done')

# Load training data

In [ ]:
# Load training data

dataset_addr = "/notebooks/INTS"

filez = list()
for (dirpath, dirnames, filenames) in os.walk(dataset_addr):
    filez += [os.path.join(dirpath, file) for file in filenames]
print('=' * 70)

filez.sort()

print('Loading training data... Please wait...')

train_data = torch.Tensor()

for f in tqdm.tqdm(filez):
    train_data = torch.cat((train_data, torch.Tensor(pickle.load(open(f, 'rb')))))
    print('Loaded file:', f)
    
print('Done!')

In [ ]:
len(train_data)

In [ ]:
len(train_data) / 4096 / 4 / 4

In [ ]:
train_data[:15], train_data[-15:]

# Setup model

In [ ]:
# Setup model

# constants

SEQ_LEN = 4096
BATCH_SIZE = 4

NUM_EPOCHS = 4

NUM_BATCHES = (len(train_data) // SEQ_LEN // BATCH_SIZE // 4) * NUM_EPOCHS

GRADIENT_ACCUMULATE_EVERY = 4
LEARNING_RATE = 2e-4

VALIDATE_EVERY  = 100
SAVE_EVERY = 1000
GENERATE_EVERY  = 200
PRINT_STATS_EVERY = 50

GENERATE_LENGTH = 32

# helpers

def cycle(loader):
    while True:
        for data in loader:
            yield data

# instantiate the model

model = LocalTransformer(
    num_tokens = 2831,
    dim = 1024,
    depth = 32,
    causal = True,
    local_attn_window_size = 512,
    max_seq_len = SEQ_LEN
).cuda()

print('Done!')
      
summary(model)

# Dataloader

class MusicDataset(Dataset):
    def __init__(self, data, seq_len):
        super().__init__()
        self.data = data
        self.seq_len = seq_len

    def __getitem__(self, index):
        
        # random sampling
        
        if index < (self.data.size(0) // self.seq_len):
            idx = index * self.seq_len
        else:
            idx = secrets.randbelow(self.data.size(0) - self.seq_len - 1)
            
        #if index % 10000 == 0:
        #    print(index, idx)
   
        full_seq = self.data[idx: idx + self.seq_len + 1].long()
        
        return full_seq.cuda()

    def __len__(self):
        return self.data.size(0)

train_dataset = MusicDataset(train_data, SEQ_LEN)
val_dataset   = MusicDataset(train_data, SEQ_LEN)
train_loader  = cycle(DataLoader(train_dataset, batch_size = BATCH_SIZE))
val_loader    = cycle(DataLoader(val_dataset, batch_size = BATCH_SIZE))

# optimizer

optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
train_dataset[666]

# Train

In [ ]:
# Train the model

train_losses = []
val_losses = []

train_accs = []
val_accs = []

for i in tqdm.tqdm(range(NUM_BATCHES), mininterval=10., desc='Training'):
    model.train()

    for __ in range(GRADIENT_ACCUMULATE_EVERY):
        loss, acc = model(next(train_loader), return_loss = True)
        loss.backward()
        
    if i % PRINT_STATS_EVERY == 0:
        print(f'Training loss: {loss.item()}')
        print(f'Training acc: {acc.item()}')
    
    train_losses.append(loss.item())
    train_accs.append(acc.item())
    
    torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
    optim.step()
    optim.zero_grad()

    if i % VALIDATE_EVERY == 0:
        model.eval()
        with torch.no_grad():
            val_loss, val_acc = model(next(val_loader), return_loss = True)
            
            print(f'Validation loss: {val_loss.item()}')
            print(f'Validation acc: {val_acc.item()}')
            
            val_losses.append(val_loss.item())
            val_accs.append(val_acc.item())
            
            print('Plotting training loss graph...')
            
            tr_loss_list = train_losses
            plt.plot([i for i in range(len(tr_loss_list))] ,tr_loss_list, 'b')
            plt.show()
            plt.close()
            print('Done!')
            
            print('Plotting training acc graph...')
            
            tr_loss_list = train_accs
            plt.plot([i for i in range(len(tr_loss_list))] ,tr_loss_list, 'b')
            plt.show()
            plt.close()
            print('Done!')
            
            print('Plotting validation loss graph...')
            tr_loss_list = val_losses
            plt.plot([i for i in range(len(tr_loss_list))] ,tr_loss_list, 'b')
            plt.show()
            plt.close()
            print('Done!')
            
            print('Plotting validation acc graph...')
            tr_loss_list = val_accs
            plt.plot([i for i in range(len(tr_loss_list))] ,tr_loss_list, 'b')
            plt.show()
            plt.close()
            print('Done!')

    if i % GENERATE_EVERY == 0:
        model.eval()
        inp = random.choice(val_dataset)[:-1]
        
        print(inp)

        sample = model.generate(inp[None, ...], GENERATE_LENGTH)
        
        print(sample)
        
    if i % SAVE_EVERY == 0:
        
        print('Saving model progress. Please wait...')
        print('model_checkpoint_' + str(i) + '_steps_' + str(round(float(train_losses[-1]), 4)) + '_loss.pth')
        
        fname = '/notebooks/model_checkpoint_'  + str(i) + '_steps_' + str(round(float(train_losses[-1]), 4)) + '_loss.pth'
        
        torch.save(model.state_dict(), fname)
        
        data = [train_losses, train_accs, val_losses, val_accs]

        TMIDIX.Tegridy_Any_Pickle_File_Writer(data, '/notebooks/losses_accs')
        
        print('Done!')      

In [ ]:
for p in val_loader:
    pass

    

# Save

In [ ]:
print('Saving model progress. Please wait...')
print('model_checkpoint_' + str(i) + '_steps_' + str(round(float(train_losses[-1]), 4)) + '_loss.pth')

fname = '/notebooks/model_checkpoint_'  + str(i) + '_steps_' + str(round(float(train_losses[-1]), 4)) + '_loss.pth'

torch.save(model.state_dict(), fname)

print('Done!')   

In [ ]:
# Save training loss graph

plt.plot([i for i in range(len(train_losses))] ,train_losses, 'b')
plt.savefig('/notebooks/training_loss_graph.png')
plt.close()
print('Done!')

# Save training acc graph

plt.plot([i for i in range(len(train_accs))] ,train_accs, 'b')
plt.savefig('/notebooks/training_acc_graph.png')
plt.close()
print('Done!')

# Save validation loss graph

plt.plot([i for i in range(len(val_losses))] ,val_losses, 'b')
plt.savefig('/notebooks/validation_loss_graph.png')
plt.close()
print('Done!')

# Save validation acc graph

plt.plot([i for i in range(len(val_accs))] ,val_accs, 'b')
plt.savefig('/notebooks/validation_acc_graph.png')
plt.close()
print('Done!')

In [ ]:
data = [train_losses, train_accs, val_losses, val_accs]

TMIDIX.Tegridy_Any_Pickle_File_Writer(data, '/notebooks/losses_accs')

# Congrats! You did it! :)

In [ ]:
model.eval()
inp = [600] * 1024 #random.choice(val_dataset)[:-1]

#inp = inp.tolist()

inp = [inp] * 8

inp = torch.LongTensor(inp).cuda()

print(inp)

sample = model.generate(inp, 1024)

print(sample)